In [7]:
!pip install -q openai langchain langchain-openai langchain_community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 18.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 3.0 MB/s eta 0:00:00


In [37]:
#API KEY 저장을 위한 os 라이브러리 호출
import os

#기본 LLM 로드를 위한 라이브러리 호출
from langchain.llms import OpenAI
# from langchain_openai.llms import OpenAI
# from langchain_community.llms import OpenAI
# from langchain_community.llms.openai import OpenAI
# from langchain_community.chat_models import ChatOpenAI


#채팅 LLM 로드를 위한 라이브러리 호출
# from langchain.chat_models import ChatOpenAI
from langchain_community.chat_models import ChatOpenAI

In [9]:
#OPENAI API키 저장
os.environ["OPENAI_API_KEY"] = 'sk-bEG4v-N0uNWGuh7Is6rHiu9TVInF7ECw3dvkU60T7bT3BlbkFJu9g9S3iyqRBkw8J6hOG6K5Sei7AIQZKslb4KLsn_UA'

-------

#### Davinch-003 모델 설정하기

In [49]:
# text-davinci-003 > gpt-3.5-turbo-instruct
davinch3 = OpenAI(
    model_name="gpt-3.5-turbo-instruct",
    max_tokens = 1000
)

#### (1) 프롬프트 템플릿 맛보기

프롬프트 템플릿은 크게 2가지가 존재합니다.
1. Prompt Template
2. Chat Prompt Template

1번 Prompt Template은 일반적인 프롬프트 템플릿을 생성할때 활용합니다.

2번 Chat Prompt Template은 채팅 LLM에 프롬프트를 전달하는 데에 활용할 수 있는 특화 프롬프트 템플릿입니다.



In [39]:
from langchain.prompts import PromptTemplate, ChatPromptTemplate

#프롬프트 템플릿을 통해 매개변수 삽입 가능한 문자열로 변환
string_prompt = PromptTemplate.from_template("tell me a joke about {subject}")

#매개변수 삽입한 결과를 string_prompt_value에 할당
string_prompt_value = string_prompt.format_prompt(subject="soccer")

#채팅LLM이 아닌 LLM과 대화할 때 필요한 프롬프트 = string prompt
string_prompt_value

StringPromptValue(text='tell me a joke about soccer')

In [40]:
#to_string() 함수를 통해 prompt template으로 생성한 문장 raw_text 반환 가능
print(string_prompt_value.to_string())

tell me a joke about soccer


In [41]:
chat_prompt = ChatPromptTemplate.from_template("tell me a joke about {subject}")
chat_prompt_value = chat_prompt.format_prompt(subject="soccer")
chat_prompt_value

ChatPromptValue(messages=[HumanMessage(content='tell me a joke about soccer')])

In [42]:
chat_prompt_value.to_string()

'Human: tell me a joke about soccer'

#### (2) 프롬프트 템플릿 활용해보기

반복적인 프롬프트를 삽입해야하는 경우, Prompt Template를 통해 간편하게 LLM을 활용할 수 있습니다.

- GPT-3와 프롬프트 템플릿을 활용하여 대화해보기

In [43]:
from langchain.prompts.prompt import PromptTemplate

template = """
너는 요리사야. 내가 가진 재료들을 갖고 만들 수 있는 요리를 추천하고, 그 요리의 레시피를 제시해줘.
내가 가진 재료는 아래와 같아.

<재료>
{재료}

"""
prompt_template = PromptTemplate(
    input_variables = ['재료'],
    template = template
)


In [44]:
print(prompt_template.format(재료 = '양파, 계란, 사과, 빵'))


너는 요리사야. 내가 가진 재료들을 갖고 만들 수 있는 요리를 추천하고, 그 요리의 레시피를 제시해줘.
내가 가진 재료는 아래와 같아.

<재료>
양파, 계란, 사과, 빵




In [50]:
print(davinch3(
    prompt_template.format(
        재료 = '양파, 계란, 사과, 빵'
    )
))

네가 만들 수 있는 요리 중 추천해줄게.

오픈페이스 샌드위치

1. 양파를 채 썰어 노릇하게 볶은 후 계란과 함께 볶는다.
2. 사과를 껍질을 벗겨서 크기에 따라 썰어준다.
3. 빵을 토스트기에 넣어 구워준다.
4. 구워진 빵 위에 양파와 계란을 올리고 그 위에 사과를 얹는다.
5. 마지막으로 빵을 덮고, 샌드위치를 완성한다.

*추가적으로 사과를 조금 더 노릇하게 볶아서 샌드위치에 올려줘도 맛있어!


-  ChatGPT와 프롬프트 템플릿을 활용하여 대화해보기


In [46]:
from langchain.prompts import (
    ChatPromptTemplate,
    PromptTemplate,
    SystemMessagePromptTemplate,
    AIMessagePromptTemplate,
    HumanMessagePromptTemplate,
)
from langchain.schema import (
    AIMessage,
    HumanMessage,
    SystemMessage
)

In [51]:
# ChatGPT 모델을 로드합니다.
chatgpt = ChatOpenAI(temperature=0)

#ChatGPT에게 역할을 부여합니다.(위에서 정의한 Template 사용)
system_message_prompt = SystemMessagePromptTemplate.from_template(template)

#사용자가 입력할 매개변수 template을 선언합니다.
human_template = "{재료}"
human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)

#ChatPromptTemplate에 system message와 human message 템플릿을 삽입합니다.
chat_prompt = ChatPromptTemplate.from_messages([system_message_prompt, human_message_prompt])

#ChatGPT API에 ChatPromptTemplate을 입력할 때, human message의 매개변수인 '재료'를 할당하여 전달합니다.
#이와 같은 방식을 통해 ChatGPT는 ChatPromptTemplate의 구성요소인 system message와 human message를 전달받아, 대답 생성에 활용합니다.
answer = chatgpt(chat_prompt.format_prompt(재료="양파, 계란, 사과, 빵").to_messages())
print(answer.content)

제가 추천하는 요리는 "양파 계란 토스트"입니다. 이 요리는 간단하면서도 맛있는 아침 식사나 간식으로 딱 좋아요. 준비물과 레시피는 아래와 같습니다.

<양파 계란 토스트>

<재료>
- 빵 4조각
- 계란 2개
- 양파 1개
- 소금
- 후추
- 식용유

<레시피>
1. 양파를 얇게 슬라이스해줍니다.
2. 팬에 식용유를 두르고 양파를 볶아 투명해질 때까지 볶아줍니다.
3. 볶은 양파를 접시에 옮겨놓고 팬을 세척하지 않습니다.
4. 계란 2개에 소금과 후추를 넣고 푹푹 섞어줍니다.
5. 팬에 계란을 부어 계란지단을 만들어줍니다.
6. 계란지단 위에 볶은 양파를 올리고 빵 한 조각을 얹어줍니다.
7. 빵이 바삭해질 때까지 노릇노릇 구워주면 완성입니다.

양파 계란 토스트는 부드러운 계란과 고소한 양파가 만나 맛있는 조합을 이루어요. 간단하게 만들 수 있으니 한 번 시도해보세요!


###(3) Few-shot 예제를 통한 프롬프트 템플릿

Few-shot이란, 딥러닝 모델이 결과물을 출력할 때 예시 결과물을 제시함으로써 원하는 결과물로 유도하는 방법론입니다.

LLM 역시, Few-shot 예제를 제공하면 예제와 유사한 형태의 결과물을 출력합니다.

내가 원하는 결과물의 형태가 특수하거나, 구조화된 답변을 원할 경우, 결과물의 예시를 수 개 제시함으로써 결과물의 품질을 향상시킬 수 있습니다.

In [52]:
from langchain.prompts.few_shot import FewShotPromptTemplate
from langchain.prompts.prompt import PromptTemplate

examples = [
  {
    "question": "아이유로 삼행시 만들어줘",
    "answer":
"""
아: 아이유는
이: 이런 강의를 들을 이
유: 유가 없다.
"""
  },

  {
    "question": "김민수로 삼행시 만들어줘",
    "answer":
"""
김: 김치는 맛있다
민: 민달팽이도 좋아하는 김치!
수: 수억을 줘도 김치는 내꺼!
"""
  }
]

In [53]:
example_prompt = PromptTemplate(input_variables=["question", "answer"], template="Question: {question}\n{answer}")

print(example_prompt.format(**examples[0]))

Question: 아이유로 삼행시 만들어줘

아: 아이유는
이: 이런 강의를 들을 이
유: 유가 없다.



In [54]:
prompt = FewShotPromptTemplate(
    examples=examples,
    example_prompt=example_prompt,
    suffix="Question: {input}",
    input_variables=["input"]
)

print(prompt.format(input="호날두로 삼행시 만들어줘"))

Question: 아이유로 삼행시 만들어줘

아: 아이유는
이: 이런 강의를 들을 이
유: 유가 없다.


Question: 김민수로 삼행시 만들어줘

김: 김치는 맛있다
민: 민달팽이도 좋아하는 김치!
수: 수억을 줘도 김치는 내꺼!


Question: 호날두로 삼행시 만들어줘


In [55]:
print(davinch3.predict("호날두로 삼행시 만들어줘"))

<ipython-input-55-d6ffda23d588>:1: LangChainDeprecationWarning: The method `BaseLLM.predict` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use invoke instead.
  print(davinch3.predict("호날두로 삼행시 만들어줘"))




멋있는 선수로 불리는

포르투갈 출신의 남자

그 이름은 크리스티아노

모든 팬들이 알고 있지

리얼 마드리드에서 빛나던

선장으로 팀을 이끌었지

오랜 시간을 뛰며 우리를 즐겁게 해줬어

근데 이번에는 유벤투스로 떠나네

그래도 우리는 그를 영원히 응원할게

그의 활약은 우리 마음속에 영원히 남아있어

그래서 이번 삼행시는 그에게 바친다

크리스티아누, 우리의 영웅이여, 감사해

소중한 추억들을 만들어줘서, 항상 빛나는 별이 되어줘서, 고마워!


In [56]:
print(davinch3(
    prompt.format(input="호날두로 삼행시 만들어줘")
))



호: 호랑이는 우렁찬데
날: 날마다 꾸는 꿈은
두: 두려움 없는 축구선수가 되는 거야


### (4) Example Selector를 이용한 동적 Few-shot 러닝

Few-shot 예제를 동적으로 입력하고 싶은 경우, Example Selector를 활용할 수 있습니다.

LLM이 여러 작업을 수행하도록 만들되 내가 원하는 범위의 대답을 출력하도록 하려면 사용자의 입력에 동적으로 반응해야 합니다.이와 동시에, 예제를 모두 학습시키는 것이 아니라 적절한 예시만 포함하도록 함으로써 입력 prompt의 길이를 제한하고, 이를 통해 오류가 발생하지 않도록 조절할 수 있습니다.

In [57]:
from langchain.prompts.example_selector import SemanticSimilarityExampleSelector
from langchain.vectorstores import Chroma
from langchain.embeddings import OpenAIEmbeddings
from langchain.prompts import FewShotPromptTemplate, PromptTemplate

example_prompt = PromptTemplate(
    input_variables=["input", "output"],
    template="Input: {input}\nOutput: {output}",
)

# These are a lot of examples of a pretend task of creating antonyms.
examples = [
    {"input": "행복", "output": "슬픔"},
    {"input": "흥미", "output": "지루"},
    {"input": "불안", "output": "안정"},
    {"input": "긴 기차", "output": "짧은 기차"},
    {"input": "큰 공", "output": "작은 공"},
]

In [58]:
!pip install chromadb
!pip install tiktoken

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 3.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 584.3/584.3 kB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 41.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 273.8/273.8 kB 18.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.5/93.5 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 64.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.6/67.6 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.2/13.2 MB 60.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.0/64.0 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.5/52.5 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.7/149.7 kB 10.9 MB/s eta 0:00:

In [59]:
example_selector = SemanticSimilarityExampleSelector.from_examples(
    # This is the list of examples available to select from.
    examples,
    # This is the embedding class used to produce embeddings which are used to measure semantic similarity.
    OpenAIEmbeddings(),
    # This is the VectorStore class that is used to store the embeddings and do a similarity search over.
    Chroma,
    # This is the number of examples to produce.
    k=1
)
similar_prompt = FewShotPromptTemplate(
    # We provide an ExampleSelector instead of examples.
    example_selector=example_selector,
    example_prompt=example_prompt,
    prefix="주어진 입력에 대해 반대의 의미를 가진 단어를 출력해줘",
    suffix="Input: {단어}\nOutput:",
    input_variables=["단어"],
)

<ipython-input-59-213a6f502b3d>:5: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  OpenAIEmbeddings(),


In [60]:
# Input is a feeling, so should select the happy/sad example
print(similar_prompt.format(단어="무서운"))

주어진 입력에 대해 반대의 의미를 가진 단어를 출력해줘

Input: 불안
Output: 안정

Input: 무서운
Output:


In [61]:
# Input is a feeling, so should select the happy/sad example
print(similar_prompt.format(단어="큰 비행기"))

주어진 입력에 대해 반대의 의미를 가진 단어를 출력해줘

Input: 긴 기차
Output: 짧은 기차

Input: 큰 비행기
Output:


In [62]:
query = "큰 비행기"

print(davinch3(
    similar_prompt.format(단어=query)
))

 작은 비행기


### (5) Output Parser를 활용한 출력값 조정

LLM의 답변을 내가 원하는 형태로 고정하고 싶다면 OutputParser 함수를 활용할 수 있습니다. 리스트, JSON 형태 등 다양한 형식의 답변을 고정하여 출력할 수 있습니다.

In [63]:
from langchain.output_parsers import CommaSeparatedListOutputParser
from langchain.prompts import PromptTemplate, ChatPromptTemplate, HumanMessagePromptTemplate
from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI

In [64]:
output_parser = CommaSeparatedListOutputParser()

In [65]:
format_instructions = output_parser.get_format_instructions()
format_instructions

'Your response should be a list of comma separated values, eg: `foo, bar, baz` or `foo,bar,baz`'

In [66]:
prompt = PromptTemplate(
    template="{주제} 5개를 추천해줘.\n{format_instructions}",
    input_variables=["주제"],
    partial_variables={"format_instructions": format_instructions}
)

In [67]:
model = OpenAI(temperature=0)

In [68]:
_input = prompt.format(주제="영화")
output = model(_input)

In [69]:
output

'\n\n1. Parasite\n2. The Shawshank Redemption\n3. Inception\n4. The Godfather\n5. Pulp Fiction'

In [70]:
output_parser.parse(output)

['1. Parasite\n2. The Shawshank Redemption\n3. Inception\n4. The Godfather\n5. Pulp Fiction']